In [2]:
%matplotlib inline
from nansat import Nansat
import numpy as np
from openwind.wind_data import fetch_era5_data, preprocess_era5_data
from datetime import timedelta
import matplotlib.pyplot as plt
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [3]:
# Step 1: Define search parameters (time and coordinates) to find ERA5 field
# In case of collocation with Sentinel-1 data these parameters can be extracted from the 
# Sentinel-1 product metadata.

# Read a Sentinel-1 L1 GRD product using Nansat
s1_frame = Nansat('/src/.devcontainer/S1A_IW_GRDH_1SDV_20220110T173429_20220110T173454_041401_04EC3B_31E4.SAFE')
# In order to speed up the processing we will resample the Sentinel-1 product to a pixel size of about 200 m
s1_frame.resize(0.05, resample_alg=1)

0.05

In [4]:
s1_lons, s1_lats = s1_frame.get_geolocation_grids()
central_lon = np.nanmean(s1_lons)
central_lat = np.nanmean(s1_lats)

In [5]:
s1_time = s1_frame.time_coverage_start
s1_time = s1_time if s1_time.minute <= 30 else s1_time + timedelta(hours=1)

In [6]:
# Find and download ERA5 data
era5_uri = fetch_era5_data(s1_time, central_lat, central_lon, '/src/.devcontainer/')

>> Downloading /src/.devcontainer/ERA5_20220110T1800.nc


2022-09-29 07:21:52,828 INFO Welcome to the CDS
2022-09-29 07:21:52,834 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-09-29 07:21:53,044 INFO Request is completed
2022-09-29 07:21:53,048 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data1/adaptor.mars.internal-1664359637.4650042-4907-10-a34c7353-d42c-4042-be15-d603029f21ff.nc to /src/.devcontainer/ERA5_20220110T1800.nc (8.2K)
2022-09-29 07:21:53,798 INFO Download rate 11K/s


In [25]:
# Derive wind speed and direction from the model
wind_spd, wind_dir = preprocess_era5(era5_uri, dst_geometry=s1_frame)

>> Processing /src/.devcontainer/ERA5_20220110T1800.nc


/opt/conda/lib/python3.7/site-packages/nansat-1.5.2-py3.7-linux-x86_64.egg/nansat/mappers/mapper_netcdf_cf.py:390: UserWarning: GDAL cannot determine the dataset projection - using Nansat spatial reference WKT, assuming a regular longitude/latitude grid
  warnings.warn('GDAL cannot determine the dataset projection - using Nansat ' \
/opt/conda/lib/python3.7/site-packages/nansat-1.5.2-py3.7-linux-x86_64.egg/nansat/mappers/mapper_netcdf_cf.py:390: UserWarning: GDAL cannot determine the dataset projection - using Nansat spatial reference WKT, assuming a regular longitude/latitude grid
  warnings.warn('GDAL cannot determine the dataset projection - using Nansat ' \


In [8]:
n = Nansat(str(era5_uri))

/opt/conda/lib/python3.7/site-packages/nansat-1.5.2-py3.7-linux-x86_64.egg/nansat/mappers/mapper_netcdf_cf.py:390: UserWarning: GDAL cannot determine the dataset projection - using Nansat spatial reference WKT, assuming a regular longitude/latitude grid
  warnings.warn('GDAL cannot determine the dataset projection - using Nansat ' \
/opt/conda/lib/python3.7/site-packages/nansat-1.5.2-py3.7-linux-x86_64.egg/nansat/mappers/mapper_netcdf_cf.py:390: UserWarning: GDAL cannot determine the dataset projection - using Nansat spatial reference WKT, assuming a regular longitude/latitude grid
  warnings.warn('GDAL cannot determine the dataset projection - using Nansat ' \


In [10]:
n.get_metadata()

{'Conventions': 'CF-1.6',
 'filename': '/vsimem/6DMOV8E5OF.vrt',
 'history': '2022-09-28 10:07:17 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data1/adaptor.mars.internal-1664359637.4650042-4907-10-a34c7353-d42c-4042-be15-d603029f21ff.nc /cache/tmp/a34c7353-d42c-4042-be15-d603029f21ff-adaptor.mars.internal-1664359634.6652615-4907-11-tmp.grib'}

In [11]:
import netCDF4 as nc

In [12]:
nc.Dataset(era5_uri)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2022-09-28 10:07:17 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data1/adaptor.mars.internal-1664359637.4650042-4907-10-a34c7353-d42c-4042-be15-d603029f21ff.nc /cache/tmp/a34c7353-d42c-4042-be15-d603029f21ff-adaptor.mars.internal-1664359634.6652615-4907-11-tmp.grib
    dimensions(sizes): longitude(41), latitude(41), time(1)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 u10(time, latitude, longitude), int16 v10(time, latitude, longitude)
    groups: 